In [67]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
from keras.utils import Sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from keras_preprocessing.image import ImageDataGenerator

import time
import pickle

#### If using AMD GPU, switch backend to PlaidML library:

In [68]:
import os
os.environ['KERAS_BACKEND']='plaidml.keras.backend'

# When using plaidml, the libraries are imported from keras instead of tensorflow
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

#### Define Experiments

In [69]:
conv_layers = [1, 2, 3]      # number of conv layers
layer_sizes = [32, 64, 128]  # number of nodes in a layer
dense_layers = [0, 1, 2]     # number of dense layers

#### Load input data

In [71]:
pickle_in = open('../Dataset/df_1pct.pickle', 'rb')
df_train, df_test = pickle.load(pickle_in)

In [73]:
# The Keras ImageDataGenerator uses string type data label
df_train['gender'] = df_train.gender.astype(str)
df_test['gender'] = df_test.gender.astype(str)

In [72]:
print(df_train.shape, df_test.shape)

(4297, 10) (226, 10)


In [75]:
df_train.dtypes

path                object
id                  uint16
name                object
dob         datetime64[ns]
gender              object
score1             float64
score2             float64
pic_date    datetime64[ns]
region              object
age                float64
dtype: object

We will be using a generator to feed model with images, the X would be the path to these images. y will be the gender label.

In [76]:
image_reshape_size = 100
input_image_root_dir = '../Dataset/imdb_crop/' # Don't forget the ending slash

In [12]:
X_train = input_image_root_dir + df_train.path
X_test = input_image_root_dir + df_test.path 
y_train = df_train.gender
y_test = df_test.gender

In [13]:
inputShape = (image_reshape_size, image_reshape_size, 1)
batch_size = 32
num_training_samples = X_train.size
num_validation_samples = X_test.size

In [14]:
df_train.head(3)

,path,id,name,dob,gender,score1,score2,pic_date,region,age
351096,67/nm0755267_rm3707735808_1980-4-8_2001.jpg,10591,Katee Sackhoff,1980-04-08,0,3.704906,3.597096,2001-01-01,"[159.42317678741446, 51.63402111024036, 209.90...",20.734170
436852,25/nm0090225_rm1239005952_1964-10-25_2007.jpg,13481,Michael Boatman,1964-10-25,1,2.629246,1.647017,2007-01-01,"[80.61172611750719, 114.87418016786741, 131.33...",42.185671
115652,18/nm0001518_rm2982264064_1961-10-26_2011.jpg,5456,Dylan McDermott,1961-10-26,1,2.793912,1.419481,2011-01-01,"[446.31, 70.47, 570.72, 194.88000000000002]",49.183761


#### Set up input image generator using flow_from_dataframe

In [79]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25)

train_generator = datagen.flow_from_dataframe(dataframe=df_train,
                                            directory=input_image_root_dir,
                                            x_col="path", y_col="gender",
                                            subset="training",
                                            class_mode="binary",
                                            color_mode="grayscale",
                                            target_size=(image_reshape_size,image_reshape_size),
                                            batch_size=32,
                                            seed=1,
                                            shuffle=True)

val_generator = datagen.flow_from_dataframe(dataframe=df_train,
                                            directory=input_image_root_dir,
                                            x_col="path", y_col="gender",
                                            subset="validation",
                                            class_mode="binary",
                                            color_mode="grayscale",
                                            target_size=(image_reshape_size,image_reshape_size),
                                            batch_size=32,
                                            seed=1,
                                            shuffle=True)

test_generator = datagen.flow_from_dataframe(dataframe=df_test, 
                                            directory=input_image_root_dir, 
                                            x_col="path", y_col=None, 
                                            class_mode=None, 
                                            color_mode="grayscale",
                                            target_size=(image_reshape_size,image_reshape_size),
                                            batch_size=1,
                                            shuffle=False)

Found 4297 validated image filenames belonging to 2 classes.
Found 226 validated image filenames belonging to 2 classes.


#### Run all training experiments

In [82]:
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            
            NAME = 'gender-{}-conv-{}-node-{}-dens-{}'.format(conv_layer, layer_size, dense_layer, int(time.time()))  # model name with timestamp
            print(NAME) 
            
            tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
            callbacks = [tensorboard]
            
            model = Sequential()
            
            # first layer
            model.add(Conv2D(layer_size, (3,3), input_shape=inputShape))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2,2)))
            
            # sets up additional # of conv layers
            for _ in range(conv_layer - 1):
                model.add(Conv2D(layer_size, (3,3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2,2)))
            
            model.add(Flatten())
            
            # sets up # of dense layers
            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))
            
            # output layer
            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            model.compile(loss='binary_crossentropy', 
                          optimizer='adam',
                          metrics=['accuracy'])

            model.fit_generator(generator=train_generator,
                                steps_per_epoch=(train_generator.n // train_generator.batch_size),
                                callbacks = callbacks,
                                validation_data=val_generator,
                                validation_steps=(val_generator.n // test_generator.batch_size),
                                epochs=10,
                                use_multiprocessing=False,
                                workers=1)

gender-1-conv-32-node-0-dens-1552855187
Epoch 1/10
134/134 [==============================] - 31s 229ms/step - loss: 0.6760 - acc: 0.5851 - val_loss: 0.6339 - val_acc: 0.6875
Epoch 2/10
134/134 [==============================] - 27s 199ms/step - loss: 0.6129 - acc: 0.6699 - val_loss: 0.6094 - val_acc: 0.6546
Epoch 3/10
134/134 [==============================] - 25s 187ms/step - loss: 0.5536 - acc: 0.7349 - val_loss: 0.5886 - val_acc: 0.6701
Epoch 4/10
134/134 [==============================] - 25s 189ms/step - loss: 0.4964 - acc: 0.7796 - val_loss: 0.5889 - val_acc: 0.6856
Epoch 5/10
134/134 [==============================] - 26s 197ms/step - loss: 0.4425 - acc: 0.8133 - val_loss: 0.6596 - val_acc: 0.6392
Epoch 6/10
134/134 [==============================] - 22s 162ms/step - loss: 0.3987 - acc: 0.8479 - val_loss: 0.6065 - val_acc: 0.6443
Epoch 7/10
134/134 [==============================] - 22s 161ms/step - loss: 0.3583 - acc: 0.8660 - val_loss: 0.7306 - val_acc: 0.6031
Epoch 8/10
134/

Epoch 10/10
134/134 [==============================] - 83s 619ms/step - loss: 0.4574 - acc: 0.7868 - val_loss: 0.6715 - val_acc: 0.6753
gender-1-conv-128-node-0-dens-1552858036
Epoch 1/10
134/134 [==============================] - 83s 618ms/step - loss: 0.7233 - acc: 0.5698 - val_loss: 0.6259 - val_acc: 0.7113
Epoch 2/10
134/134 [==============================] - 83s 620ms/step - loss: 0.6030 - acc: 0.6836 - val_loss: 0.6081 - val_acc: 0.6495
Epoch 3/10
134/134 [==============================] - 83s 619ms/step - loss: 0.5212 - acc: 0.7571 - val_loss: 0.6079 - val_acc: 0.6546
Epoch 4/10
134/134 [==============================] - 82s 610ms/step - loss: 0.4552 - acc: 0.8006 - val_loss: 0.6771 - val_acc: 0.5773
Epoch 5/10
134/134 [==============================] - 83s 621ms/step - loss: 0.3821 - acc: 0.8485 - val_loss: 0.6186 - val_acc: 0.6753
Epoch 6/10
134/134 [==============================] - 82s 615ms/step - loss: 0.3091 - acc: 0.8781 - val_loss: 0.7189 - val_acc: 0.6340
Epoch 7/10
13

134/134 [==============================] - 35s 262ms/step - loss: 0.4874 - acc: 0.7749 - val_loss: 0.6374 - val_acc: 0.6237
Epoch 10/10
134/134 [==============================] - 35s 259ms/step - loss: 0.4468 - acc: 0.7942 - val_loss: 0.7293 - val_acc: 0.6786
gender-1-conv-64-node-1-dens-1552863776
Epoch 1/10
134/134 [==============================] - 45s 336ms/step - loss: 0.7612 - acc: 0.6009 - val_loss: 0.6322 - val_acc: 0.6701
Epoch 2/10
134/134 [==============================] - 48s 360ms/step - loss: 0.5651 - acc: 0.7135 - val_loss: 0.6100 - val_acc: 0.6473
Epoch 3/10
134/134 [==============================] - 47s 352ms/step - loss: 0.4356 - acc: 0.8058 - val_loss: 0.6491 - val_acc: 0.6289
Epoch 4/10
134/134 [==============================] - 46s 347ms/step - loss: 0.3072 - acc: 0.8716 - val_loss: 0.6988 - val_acc: 0.6495
Epoch 5/10
134/134 [==============================] - 48s 360ms/step - loss: 0.1911 - acc: 0.9346 - val_loss: 0.8949 - val_acc: 0.5722
Epoch 6/10
134/134 [=====

134/134 [==============================] - 203s 2s/step - loss: 0.4129 - acc: 0.8063 - val_loss: 0.7014 - val_acc: 0.6856
Epoch 9/10
134/134 [==============================] - 203s 2s/step - loss: 0.3303 - acc: 0.8598 - val_loss: 0.8241 - val_acc: 0.6598
Epoch 10/10
134/134 [==============================] - 203s 2s/step - loss: 0.2247 - acc: 0.9113 - val_loss: 1.0009 - val_acc: 0.6546
gender-1-conv-32-node-2-dens-1552871519
Epoch 1/10
134/134 [==============================] - 24s 177ms/step - loss: 0.6935 - acc: 0.5709 - val_loss: 0.6581 - val_acc: 0.6134
Epoch 2/10
134/134 [==============================] - 19s 139ms/step - loss: 0.6382 - acc: 0.6402 - val_loss: 0.5851 - val_acc: 0.7113
Epoch 3/10
134/134 [==============================] - 19s 140ms/step - loss: 0.5797 - acc: 0.7038 - val_loss: 0.6458 - val_acc: 0.6598
Epoch 4/10
134/134 [==============================] - 18s 137ms/step - loss: 0.5043 - acc: 0.7645 - val_loss: 0.5916 - val_acc: 0.6753
Epoch 5/10
134/134 [===========

Epoch 7/10
134/134 [==============================] - 77s 574ms/step - loss: 0.4832 - acc: 0.7695 - val_loss: 0.6750 - val_acc: 0.6959
Epoch 8/10
134/134 [==============================] - 75s 563ms/step - loss: 0.4281 - acc: 0.8045 - val_loss: 0.6917 - val_acc: 0.6959
Epoch 9/10
134/134 [==============================] - 76s 567ms/step - loss: 0.3468 - acc: 0.8450 - val_loss: 0.8097 - val_acc: 0.6082
Epoch 10/10
134/134 [==============================] - 76s 566ms/step - loss: 0.2541 - acc: 0.8905 - val_loss: 1.0211 - val_acc: 0.6649
gender-1-conv-128-node-2-dens-1552874229
Epoch 1/10
134/134 [==============================] - 150s 1s/step - loss: 0.8177 - acc: 0.5682 - val_loss: 0.6268 - val_acc: 0.6959
Epoch 2/10
134/134 [==============================] - 146s 1s/step - loss: 0.6005 - acc: 0.6878 - val_loss: 0.6377 - val_acc: 0.6340
Epoch 3/10
134/134 [==============================] - 153s 1s/step - loss: 0.4637 - acc: 0.7802 - val_loss: 0.6802 - val_acc: 0.6392
Epoch 4/10
134/134 

To view the tensorboard, use command:
tensorboard --logdir=logs/

#### Evaluation

In [ ]:
test_generator.reset()
pred=model.predict_generator(test_generator,
                            steps=test_generator.n//test_generator.batch_size,
                            verbose=1)
pred_class=np.argmax(pred,axis=1) # index of largest value in each row